In [1]:
import numpy as np
from scipy import *
import os
import h5py
import keras as k
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras import *
from astropy.io import fits

Using TensorFlow backend.


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\Logistic Regression\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

def crop_center_single(img,cut):
    y = np.shape(img)[0]
    x = np.shape(img)[1]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[start:end,start:end]

In [4]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)

In [5]:
# Reshape data

img = np.reshape(images_cut, (-1, 18225))
labels = np.reshape(labels, (-1,1))

In [6]:
##### This section is horrendous, I know #####

# Normalising the number of examples for each class
nb_FRI = int(np.asarray((np.where(labels == False))).size/2)
nb_FRII = int(np.asarray((np.where(labels == True))).size/2)

# Defining the maximum number of samples for each class
if nb_FRI >= nb_FRII:
    cutoff = nb_FRII
elif nb_FRI < nb_FRII:
        cutoff = nb_FRI

# seperating images and labels by class
FRI = img[cutoff:nb_FRI, :]
FRI_labels = labels[cutoff:nb_FRI]


In [7]:
#imgs = np.reshape(FRI, (-1, 135, 135))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(135, 135))
   #print (n)
   #print (FRI_labels[n])
   #plt.show()

In [8]:
# load fake sample
fit_file = fits.open('fr.fits')
fit_img = np.asarray(fit_file[0].data)
fit_img.shape
fit_cut = crop_center_single(fit_img, 32)
fit_shaved = fit_cut[1:,1:]
fit_flattened = np.reshape(fit_shaved, (-1, 18225))

In [20]:
# Split data into training and testing sets
test_x = FRI
test_yy = FRI_labels

In [10]:
# Convert class vectors to binary class matrices
test_y = np_utils.to_categorical(test_yy, 2)

In [16]:

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = k.models.model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
 
# evaluate loaded model on test data
sgd = SGD(lr=1)

loaded_model.compile(optimizer=sgd, loss='categorical_crossentropy',
              metrics=['accuracy'])

score = loaded_model.evaluate(test_x, test_y, verbose=0)
pred = loaded_model.predict(test_x)

In [14]:
score[1] 

0.83636363311247397

In [29]:
fit_y = np_utils.to_categorical(np.asarray([[True]]), 2)
score_fit = loaded_model.evaluate(fit_flattened, fit_y)
score_fit[1]

1/1 [==============================] - 0s 2ms/step


0.0